# Process

What is a process? A process is a program in execution.

What is a program? A program is a file containing the information of a process and how to build it during run time. When you start execution of the program, it is loaded into RAM and starts executing.

Each process is identified with a unique positive integer called as process ID or simply PID (Process Identification number). The kernel usually limits the process ID to 32767, which is configurable. When the process ID reaches this limit, it is reset again, which is after the system processes range. The unused process IDs from that counter are then assigned to newly created processes.



The system call getpid() returns the process ID of the calling process.
```bash
#include <sys/types.h>
#include <unistd.h>

pid_t getpid(void);
```
This call returns the process ID of the calling process which is guaranteed to be unique. This call is always successful and thus no return value to indicate an error.

Each process has its unique ID called process ID. Creator process is called the parent process. Parent ID or PPID can be obtained through getppid() call.
The system call getppid() returns the Parent PID of the calling process.
```bash
#include <sys/types.h>
#include <unistd.h>

pid_t getppid(void);
```
This call returns the parent process ID of the calling process. This call is always successful and thus no return value to indicate an error.

In [ ]:
%%writefile processinfo.c
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <unistd.h>

int main() {
   int mypid, myppid;
   printf("Program to know PID and PPID's information\n");
   mypid = getpid();
   myppid = getppid();
   printf("My process ID is %d\n", mypid);
   printf("My parent process ID is %d\n", myppid);
   printf("Cross verification of pid's by executing process commands on shell\n");
   system("ps -ef");
   return 0;
}

Writing processinfo.c


In [ ]:
!gcc processinfo.c -o processinfo

In [ ]:
!./processinfo

Program to know PID and PPID's information
My process ID is 307
My parent process ID is 153
Cross verification of pid's by executing process commands on shell
UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 19:11 ?        00:00:00 /sbin/docker-init -- /datalab/run.sh
root           7       1  2 19:11 ?        00:00:00 /tools/node/bin/node /datalab/web/app.js
root          18       7  0 19:11 ?        00:00:00 /bin/bash -e /usr/local/colab/bin/oom_monitor.sh
root          20       1  0 19:11 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          23      20  0 19:11 ?        00:00:00 /usr/colab/bin/kernel_manager_proxy --listen_por
root          24       0  0 19:11 ?        00:00:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/
root          64       7 22 19:11 ?        00:00:08 [python3] <defunct>
root          65       7  2 19:11 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root          87       7 10 19:11 ?        00:00:03 /u

Process creation is achieved through the fork() system call. The newly created process is called the child process and the process that initiated it (or the process when execution is started) is called the parent process. After the fork() system call, now we have two processes - parent and child processes. How to differentiate them? Very simple, it is through their return values.
```bash
#include <sys/types.h>
#include <unistd.h>

pid_t fork(void);
```
After this call, there are two processes, the existing one is called the parent process and the newly created one is called the child process.

The fork() system call returns either of the three values −
* Negative value to indicate an error, i.e., unsuccessful in creating the child process.
* Returns a zero for child process.
* Returns a positive value for the parent process. This value is the process ID of the newly created child process.

In [ ]:
%%writefile basicfork.c
#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>

int main() {
   fork();
   printf("Called fork() system call\n");
   return 0;
}

Writing basicfork.c


In [ ]:
!gcc basicfork.c -o basicfork

In [ ]:
!./basicfork

Called fork() system call
Called fork() system call


Usually after fork() call, the child process and the parent process would perform different tasks. If the same task needs to be run, then for each fork() call it would run 2 power n times, where n is the number of times fork() is invoked.

In the above case, fork() is called once, hence the output is printed twice (2 power 1). If fork() is called, say 3 times, then the output would be printed 8 times (2 power 3). If it is called 5 times, then it prints 32 times and so on and so forth.

In [ ]:
%%writefile pids_after_fork.c
#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>

int main() {
   pid_t pid, mypid, myppid;
   pid = getpid();
   printf("Before fork: Process id is %d\n", pid);
   pid = fork();

   if (pid < 0) {
      perror("fork() failure\n");
      return 1;
   }

   // Child process
   if (pid == 0) {
      printf("This is child process\n");
      mypid = getpid();
      myppid = getppid();
      printf("Process id is %d and PPID is %d\n", mypid, myppid);
   } else { // Parent process
      sleep(2);
      printf("This is parent process\n");
      mypid = getpid();
      myppid = getppid();
      printf("Process id is %d and PPID is %d\n", mypid, myppid);
      printf("Newly created process id or child pid is %d\n", pid);
   }
   return 0;
}

Writing pids_after_fork.c


In [ ]:
!gcc pids_after_fork.c -o pids_after_fork

In [ ]:
!./pids_after_fork

Before fork: Process id is 3151
This is child process
Process id is 3152 and PPID is 3151
This is parent process
Process id is 3151 and PPID is 153
Newly created process id or child pid is 3152


A process can terminate in either of the two ways:
* Abnormally, occurs on delivery of certain signals, say terminate signal.
* Normally, using _exit() system call (or _Exit() system call) or exit() library function.

The difference between _exit() and exit() is mainly the cleanup activity. The exit() does some cleanup before returning the control back to the kernel, while the _exit() (or _Exit()) would return the control back to the kernel immediately.

The atexit() function in C and C++ allows you to register a function to be called automatically when the program terminates normally.
```bash
int atexit(void (*func)(void));
```
The function returns: Zero if the function registration is successful; Non-zero if the function registration failed.
* You can register one or more functions using atexit().
* These functions will be executed in the reverse order they were registered (i.e., the last function specified is the first to be executed at exit).
* A single function can be registered to be executed at exit more than once.
* The function is called with no arguments.

In [ ]:
%%writefile atexit_sample.c
#include <stdio.h>
#include <stdlib.h>

void exitfunc() {
   printf("Called cleanup function - exitfunc()\n");
   return;
}

int main() {
   atexit(exitfunc);
   printf("Hello, World!\n");
   exit (0);
}

Writing atexit_sample.c


In [ ]:
!gcc atexit_sample.c -o atexit_sample

In [ ]:
!./atexit_sample

Hello, World!
Called cleanup function - exitfunc()


Consider the following example program with _exit().

In [ ]:
%%writefile at_exit_sample.c
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>

void exitfunc() {
   printf("Called cleanup function - exitfunc()\n");
   return;
}

int main() {
   atexit(exitfunc);
   printf("Hello, World!\n");
   _exit (0);
}

Overwriting at_exit_sample.c


In [ ]:
!gcc at_exit_sample.c -o at_exit_sample

In [ ]:
!./at_exit_sample

Hello, World!


To monitor the child process execution state, to check whether the child process is running or stopped or to check the execution status, etc. the wait() system calls and its variants is used.

In [ ]:
%%writefile parentprocess_nowait.c
#include<stdio.h>
#include <unistd.h>
#include <stdlib.h>
int main() {
   int pid;
   pid = fork();

   // Child process
   if (pid == 0) {
      system("ps -ef");
      sleep(10);
      system("ps -ef");
   } else {
      sleep(3);
   }
   return 0;
}

Overwriting parentprocess_nowait.c


In [ ]:
!gcc parentprocess_nowait.c -o parentprocess_nowait

In [ ]:
!./parentprocess_nowait

UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 19:11 ?        00:00:00 /sbin/docker-init -- /datalab/run.sh
root           7       1  0 19:11 ?        00:00:04 /tools/node/bin/node /datalab/web/app.js
root          18       7  0 19:11 ?        00:00:00 /bin/bash -e /usr/local/colab/bin/oom_monitor.sh
root          20       1  0 19:11 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          23      20  0 19:11 ?        00:00:00 /usr/colab/bin/kernel_manager_proxy --listen_por
root          24       0  0 19:11 ?        00:00:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/
root          64       7  0 19:11 ?        00:00:08 [python3] <defunct>
root          65       7  0 19:11 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root          87       7  0 19:11 ?        00:00:05 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root          88       7  0 19:11 ?        00:00:00 /usr/local/bin/dap_multiplexer --domain_socket_p
root       

Observe that the parent process PID was 5393 and the child process PID was 5394. After the parent process exits, the PPID of the child process changed from 5393 to 1 (init process).

The wait() system call would wait for one of the children to terminate and return its termination status in the buffer
```bash
#include <sys/types.h>
#include <sys/wait.h>

pid_t wait(int *status);
```
This call returns the process ID of the terminated child on success and -1 on failure. The wait() system call suspends the execution of the current process and waits indefinitely until one of its children terminates. The termination status from the child is available in status.

In [ ]:
%%writefile parentprocess_waits.c
#include<stdio.h>
#include<unistd.h>
#include<stdlib.h>
#include <sys/types.h>
#include <sys/wait.h>

int main() {
   int pid;
   int status;
   pid = fork();

   // Child process
   if (pid == 0) {
      system("ps -ef");
      sleep(10);
      system("ps -ef");
      return 3; //exit status is 3 from child process
   } else {
      sleep(3);
      wait(&status);
      printf("In parent process: exit status from child is decimal %d, hexa %0x\n", status, status);
   }
   return 0;
}

Overwriting parentprocess_waits.c


In [ ]:
!gcc parentprocess_waits.c -o parentprocess_waits

In [ ]:
!./parentprocess_waits

UID          PID    PPID  C STIME TTY          TIME CMD
root           1       0  0 19:11 ?        00:00:00 /sbin/docker-init -- /datalab/run.sh
root           7       1  0 19:11 ?        00:00:04 /tools/node/bin/node /datalab/web/app.js
root          18       7  0 19:11 ?        00:00:01 /bin/bash -e /usr/local/colab/bin/oom_monitor.sh
root          20       1  0 19:11 ?        00:00:00 /bin/bash -e /datalab/run.sh
root          23      20  0 19:11 ?        00:00:00 /usr/colab/bin/kernel_manager_proxy --listen_por
root          24       0  0 19:11 ?        00:00:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/
root          64       7  0 19:11 ?        00:00:08 [python3] <defunct>
root          65       7  0 19:11 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root          87       7  0 19:11 ?        00:00:05 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root          88       7  0 19:11 ?        00:00:01 /usr/local/bin/dap_multiplexer --domain_socket_p
root       

The waitpid() system call would wait for specified children to terminate and return its termination status in the buffer
```bash
#include <sys/types.h>
#include <sys/wait.h>

pid_t waitpid(pid_t pid, int *status, int options);
```
The above call returns the process ID of the terminated child on success and -1 on failure. The waitpid() system call suspends the execution of the current process and waits indefinitely until the specified children (as per pid value) terminates. The termination status from the child is available in the status.

The value of pid can be either of the following:
* `< -1` − Wait for any child process whose process group ID is equal to the absolute value of pid.
* `-1` − Wait for any child process, which equals to that of wait() system call.
* `0` − Wait for any child process whose process group ID is equal to that of the calling process.
* `>0` − Wait for any child process whose process ID is equal to the value of pid.

By default, waitpid() system call waits only for the terminated children but this default behavior can be modified using the options argument.

In [ ]:
%%writefile waitpid_test.c
#include<stdio.h>
#include<unistd.h>
#include<sys/types.h>
#include<sys/wait.h>

int main() {
   int pid;
   int pids[3];
   int status;
   int numprocesses = 0;
   int total_processes = 3;
   while (numprocesses < total_processes) {
      pid = fork();

      // Child process
      if (pid == 0) {
         printf("In child process: process id is %d\n", getpid());
         sleep(5);
         return 4;
      } else {
         pids[numprocesses] = pid;
         numprocesses++;
         printf("In parent process: created process number: %d\n", pid);
      }
   }

   // Waiting for 3rd child process
   waitpid(pids[total_processes - 1], &status, 0);
   if (WIFEXITED(status) != 0) {
      printf("process %d exited normally\n", pids[total_processes - 1]);
      printf("exit status from child is %d\n", WEXITSTATUS(status));
   } else {
      printf("process %d not exited normally\n", pids[total_processes - 1]);
   }
   return 0;
}

Writing waitpid_test.c


In [ ]:
!gcc waitpid_test.c -o waitpid_test

In [ ]:
!./waitpid_test

In parent process: created process number: 7866
In parent process: created process number: 7867
In parent process: created process number: 7868
In child process: process id is 7866
In child process: process id is 7867
In child process: process id is 7868
process 7868 exited normally
exit status from child is 4


# Thread

What is a Thread?  A thread is a single sequence stream within a process. Because threads have some of the properties of processes, they are sometimes called lightweight processes.

What are the differences between process and thread?  Threads are not independent from each other unlike processes. As a result, threads shares with other threads their code section, data section and OS resources like open files and signals. But, like processes, a thread has its own program counter (PC), a register set, and a stack space.

Threads are popular way to improve application through parallelism.

Threads operate faster than processes due to following reasons:
  1) Thread creation is much faster.
  2) Context switching between threads is much faster.
  3) Threads can be terminated easily
  4) Communication between threads is faster.

## Thread in C

Unlike Java, multithreading is not supported by the language standard. POSIX Threads (or Pthreads) is a POSIX standard for threads. Implementation of pthread is available with gcc compiler.

A simple C program to demonstrate use of pthread basic functions

In [ ]:
%%writefile thread.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h> //Header file for sleep(). man 3 sleep for details.
#include <pthread.h>

// A normal C function that is executed as a thread
// when its name is specified in pthread_create()
void *myThreadFun(void *vargp)
{
	sleep(1);
	printf("Printing Hello from Thread \n");
	return NULL;
}

int main()
{
	pthread_t thread_id;
	printf("Before Thread\n");
	pthread_create(&thread_id, NULL, myThreadFun, NULL);
	pthread_join(thread_id, NULL);
	printf("After Thread\n");
	exit(0);
}


Writing thread.c


In [ ]:
!gcc thread.c -o thread

In [ ]:
!./thread

Before Thread
Printing Hello from Thread 
After Thread


In main(), we declare a variable called thread_id, which is of type pthread_t, which is an integer used to identify the thread in the system. After declaring thread_id, we call pthread_create() function to create a thread.

pthread_create() takes 4 arguments.

The first argument is a pointer to thread_id which is set by this function.

The second argument specifies attributes. If the value is NULL, then default attributes shall be used.

The third argument is name of function to be executed for the thread to be created.

The fourth argument is used to pass arguments to the function, myThreadFun.

The pthread_join() function for threads is the equivalent of wait() for processes. A call to pthread_join blocks the calling thread until the thread with identifier equal to the first argument terminates.

A C program to show multiple threads with global and static variables

In [ ]:
%%writefile thread2.c
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <pthread.h>

// Let us create a global variable to change it in threads
int g = 0;

// The function to be executed by all threads
void *myThreadFun(void *vargp)
{
	// Store the value argument passed to this thread
	int *myid = (int *)vargp;

	// Let us create a static variable to observe its changes
	static int s = 0;

	// Change static and global variables
	++s; ++g;

	// Print the argument, static and global variables
	printf("Thread ID: %d, Static: %d, Global: %d\n", *myid, ++s, ++g);
}

int main()
{
	int i;
	pthread_t tid;

	// Let us create three threads
	for (i = 0; i < 3; i++)
		pthread_create(&tid, NULL, myThreadFun, (void *)&tid);

	pthread_exit(NULL);
	return 0;
}


Writing thread2.c


In [ ]:
!gcc thread2.c -o thread2

In [ ]:
!./thread2

Thread ID: 106063424, Static: 2, Global: 2
Thread ID: 106063424, Static: 4, Global: 4
Thread ID: 106063424, Static: 6, Global: 6


Accessing a global variable in a thread is generally a bad idea. What if thread 2 has priority over thread 1 and thread 1 needs to change the variable. In practice, if it is required to access global variable by multiple threads, then they should be accessed using a mutex.

## Thread in C++

Multithreading is a feature that allows concurrent execution of two or more parts of a program for maximum utilization of the CPU. Each part of such a program is called a thread. So, threads are lightweight processes within a process.

Multithreading support was introduced in C++11. Prior to C++11, we had to use POSIX threads or <pthreads> library. While this library did the job the lack of any standard language-provided feature set caused serious portability issues. C++ 11 did away with all that and gave us std::thread. The thread classes and related functions are defined in the <thread> header file.
```bash
std::thread thread_object (callable);
```

std::thread is the thread class that represents a single thread in C++. To start a thread we simply need to create a new thread object and pass the executing code to be called (i.e, a callable object) into the constructor of the object. Once the object is created a new thread is launched which will execute the code specified in callable. A callable can be any of the five:
* A Function Pointer
* A Lambda Expression
* A Function Object
* Non-Static Member Function
* Static Member Function

https://www.geeksforgeeks.org/multithreading-in-cpp/

Note that we have a new Standard C++ Library header `#include <thread>` in which the functions and classes for threads are declared.

In [ ]:
%%writefile t1.cpp
#include <iostream>
#include <thread>

void thread_function()
{
    std::cout << "thread function\n";
}

int main()
{
    std::thread t(&thread_function);   // t starts running
    std::cout << "main thread\n";
    t.join();   // main thread waits for the thread t to finish
    return 0;
}

Writing t1.cpp


In [ ]:
!g++ t1.cpp -o t1 -std=c++11 -pthread

In [ ]:
!./t1

main thread
thread function


Detaching Threads: We can make a new thread to run free to become a daemon process.

In [ ]:
%%writefile t2.cpp
#include <iostream>
#include <thread>

void thread_function()
{
    std::cout << "thread function\n";
}

int main()
{
    std::thread t(&thread_function);
    std::cout << "main thread\n";
    // t.join();
    t.detach();
    return 0;
}

Writing t2.cpp


In [ ]:
!g++ t2.cpp -o t2 -std=c++11 -pthread

In [ ]:
!./t2

main thread


Note that the detached thread didn't have a change to print its output to stdout because the main thread already finished and exited. This is one of the characteristics of multithreaded programming: we cannot be sure which thread runs first (not deterministic unless we use synchronization mechanism). In our case, because the time it takes to create a new thread, the main thread is most likely to finish ahead of our child thread.

Once a thread detached, we cannot force it to join with the main thread again.

We can keep the code from crashing by checking using joinable(). Because it's not joinable, the join() function won't be called, and the program runs without crash.

In [ ]:
%%writefile t3.cpp
#include <iostream>
#include <thread>

void thread_function()
{
    std::cout << "thread function\n";
}

int main()
{
    std::thread t(&thread_function);
    std::cout << "main thread\n";
    // t.join();
    t.detach();
    // t.join(); Error
    if(t.joinable())
        t.join();
    return 0;
}

Writing t3.cpp


In [ ]:
!g++ t3.cpp -o t3 -std=c++11 -pthread

In [ ]:
!./t3

main thread
thread function


we can use any callable object

In [ ]:
%%writefile threadcpp.cpp
// C++ program to demonstrate
// multithreading using three
// different callables.
#include <iostream>
#include <thread>
using namespace std;

// A dummy function
void foo(int Z)
{
	for (int i = 0; i < Z; i++) {
		cout << "Thread using function"
				" pointer as callable\n";
	}
}

// A callable object
class thread_obj {
public:
	void operator()(int x)
	{
		for (int i = 0; i < x; i++)
			cout << "Thread using function"
					" object as callable\n";
	}
};

// class definition
class Base {
public:
	// non-static member function
	void foo()
	{
		cout << "Thread using non-static member function "
				"as callable"
			<< endl;
	}
	// static member function
	static void foo1()
	{
		cout << "Thread using static member function as "
				"callable"
			<< endl;
	}
};

// Driver code
int main()
{
	cout << "Threads 1 and 2 and 3 "
			"operating independently"
		<< endl;

	// This thread is launched by using
	// function pointer as callable
	thread th1(foo, 3);

	// This thread is launched by using
	// function object as callable
	thread th2(thread_obj(), 3);

	// Define a Lambda Expression
	auto f = [](int x) {
		for (int i = 0; i < x; i++)
			cout << "Thread using lambda"
					" expression as callable\n";
	};

	// This thread is launched by using
	// lambda expression as callable
	thread th3(f, 3);

	// object of Base Class
	Base b;

	thread th4(&Base::foo, &b);

	thread th5(&Base::foo1);

	// Wait for the threads to finish
	// Wait for thread t1 to finish
	th1.join();

	// Wait for thread t2 to finish
	th2.join();

	// Wait for thread t3 to finish
	th3.join();

	// Wait for thread t4 to finish
	th4.join();

	// Wait for thread t5 to finish
	th5.join();

	return 0;
}


Writing threadcpp.cpp


In [ ]:
%%bash

g++ threadcpp.cpp -o threadcpp
./threadcpp

Threads 1 and 2 and 3 operating independently
Thread using function pointer as callable
Thread using function pointer as callable
Thread using function pointer as callable
Thread using non-static member function as callable
Thread using static member function as callable
Thread using function object as callable
Thread using function object as callable
Thread using function object as callable
Thread using lambda expression as callable
Thread using lambda expression as callable
Thread using lambda expression as callable


We can get hardware_concurrency information

In [ ]:
%%writefile threadinfo.cpp
#include <iostream>
#include <thread>

void thread_function()
{
    std::cout << "thread function\n";
}

int main()
{
    std::cout << "Number of threads = "
              <<  std::thread::hardware_concurrency() << std::endl;
    return 0;
}


Overwriting threadinfo.cpp


In [ ]:
!g++ threadinfo.cpp -o threadinfo -std=c++11 -pthread

In [ ]:
!./threadinfo

Number of threads = 2


## Resource race

In [ ]:
%%writefile race.cpp
#include <iostream>
#include <thread>

void thread_function()
{
    for (int i = -100; i < 0; i++)
        std::cout << "thread function: " << i << "\n";
}

int main()
{
	std::thread t(&thread_function);
	for (int i = 0; i < 100; i++)
	    std::cout << "main thread: " << i << "\n";
	t.join();
	return 0;
}

Writing race.cpp


In [ ]:
!g++ race.cpp -o race -std=c++11 -pthread

In [ ]:
!./race

main thread: 0
main thread: 1
main thread: 2
main thread: 3
main thread: 4
main thread: 5
main thread: 6
main thread: 7
main thread: 8
main thread: 9
main thread: 10
main thread: 11
main thread: 12
main thread: 13
main thread: 14
main thread: 15
main thread: 16
main thread: 17
main thread: 18
main thread: 19
main thread: 20
main thread: 21
main thread: 22
main thread: 23
main thread: 24
main thread: 25
main thread: 26
main thread: 27
main thread: 28
main thread: 29
main thread: 30
main thread: 31
main thread: 32
main thread: 33
main thread: 34
main thread: 35
main thread: 36
main thread: 37
main thread: 38
main thread: 39
main thread: 40
main thread: 41
main thread: 42
main thread: 43
main thread: 44
main thread: 45
main thread: 46
main thread: 47
main thread: 48
main thread: 49
main thread: 50
main thread: 51
main thread: 52
main thread: 53
main thread: 54
main thread: 55
main thread: 56
main thread: 57
main thread: 58
main thread: 59
main thread: 60
main thread: 61
main thread: 62
ma

As we can see from the output, the two threads get the cout resource in a ramdom fashion. To have a deterministic access, the code below is using mutex: lock before accessing cout, and then unlock after using it:

In [ ]:
%%writefile racemutex.cpp
#include <iostream>
#include <thread>
#include <string>
#include <mutex>

std::mutex mu;

void shared_cout(std::string msg, int id)
{
	mu.lock();
	std::cout << msg << ":" << id << std::endl;
	mu.unlock();
}
void thread_function()
{
	for (int i = -100; i < 0; i++)
		shared_cout("thread function", i);
}

int main()
{
	std::thread t(&thread_function);
	for (int i = 100; i > 0; i--)
	    shared_cout("main thread", i);
	t.join();
	return 0;
}

Writing racemutex.cpp


In [ ]:
!g++ racemutex.cpp -o racemutex -std=c++11 -pthread

In [ ]:
!./racemutex

main thread:100
main thread:99
main thread:98
main thread:97
main thread:96
main thread:95
main thread:94
main thread:93
main thread:92
main thread:91
main thread:90
main thread:89
main thread:88
main thread:87
main thread:86
main thread:85
main thread:84
main thread:83
main thread:82
main thread:81
main thread:80
main thread:79
main thread:78
main thread:77
main thread:76
main thread:75
main thread:74
main thread:73
main thread:72
main thread:71
main thread:70
main thread:69
main thread:68
main thread:67
main thread:66
main thread:65
main thread:64
main thread:63
main thread:62
main thread:61
main thread:60
main thread:59
main thread:58
main thread:57
main thread:56
main thread:55
main thread:54
main thread:53
main thread:52
main thread:51
main thread:50
main thread:49
main thread:48
main thread:47
main thread:46
main thread:45
main thread:44
main thread:43
main thread:42
main thread:41
main thread:40
main thread:39
main thread:38
main thread:37
main thread:36
main thread:35
main thre

Now the output becomes more in order

The issued of sharing data between threads are mostly due to the consequences of modifying data.

If the data we share is read-only data, there will be no problem, because the data read by one thread is unaffected by whether or not another thread is reading the same data. However, once data is shared between threads, and one or more threads start modifying the data, and that's the start of problems. In this case, we must take care to make sure that everything works out fine.

Though mutexes are probably the most widely used data-protection mechanism in C++, it's important to structure our code to protect the right data and avoid race conditions inherent in our interfaces. Mutexes also come with their own problems, in the form of a deadlock and protecting either too much or too little data.

In C++, we create a mutex by constructing an instance of std::mutex, lock it with a call to the member function lock(), and unlock it with a call to the member function unlock(). However, it is not a good practice to call the member functions directly, because this means that we have to remember to call unlock() on every code path out of a function, including those due to exceptions.

In other words, if we have an exception in the code after the lock() but before the unlock(), then we'll have a problem. The resources are in locked state!

So, the Standard C++ Library provides the std::lock_guard class template, which implements that RAII idiom for a mutex. It locks the supplied mutex on construction and unlocks it on destruction, thus ensuring a locked mutex is always correctly unlocked.

The example below shows how to protect a list that can be accessed by multiple threads using a std::mutex, along with std::lock_guard. Both of these are declared in the <mutex> header.

In [ ]:
%%writefile racemutex2.cpp
#include <iostream>
#include <thread>
#include <list>
#include <algorithm>
#include <mutex>

using namespace std;

// a global variable
std::list<int>myList;

// a global instance of std::mutex to protect global variable
std::mutex myMutex;

void addToList(int max, int interval)
{
	// the access to this function is mutually exclusive
	std::lock_guard<std::mutex> guard(myMutex);
	for (int i = 0; i < max; i++) {
		if( (i % interval) == 0) myList.push_back(i);
	}
}

void printList()
{
	// the access to this function is mutually exclusive
	std::lock_guard<std::mutex> guard(myMutex);
	for (auto itr = myList.begin(), end_itr = myList.end(); itr != end_itr; ++itr ) {
		cout << *itr << ",";
	}
}

int main()
{
	int max = 100;

	std::thread t1(addToList, max, 1);
	std::thread t2(addToList, max, 10);
	std::thread t3(printList);

	t1.join();
	t2.join();
	t3.join();

	return 0;
}

Writing racemutex2.cpp


In [ ]:
!g++ racemutex2.cpp -o racemutex2 -std=c++11 -pthread

In [ ]:
!./racemutex2

0,10,20,30,40,50,60,70,80,90,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,

The new output shows that we got the right process working on the list.